#### SC - Skin Conductance
SC signal se sastoji od sporo promjenjive toničke komponente
na čijem vrhu se superponira brže promjenjiva fazna komponenta

#### Fazna komponenta
- Brzo promjenjiva
- Očitava se na vrhu toničke komponente
- Sastoji se od SCR-ova (skin conductance responses)
- Svaki od SCR-ova je generiran neuralnom aktivnosti
- Stope po kojima se javljaju SCR-ovi i njihove amplitedu su markeri uzbuđenja

#### Tonička komponenta
- Sporo promjenjiva
- Povezana sa termoregulacijom
- Ovisi o temperaturi i vlažnosti okoline
- Zahvaća opće varijacije u uzbuđenju

#### Komponente SC signala
<img style="float: left;" src="Skin Conductance.png">

#### 1. korak - Niskopropusni filtar granične frekvencije (!?)
- Za izdvajanje SC trendova
- Visokopropusna filtrirana komponenta (s niskom graničnom frekvencijom) se oduzima od SC signala da bi se izdvojila tonička komponenta
- Oni koriste lowpass FIR filter - (primjer za python: https://scipy-cookbook.readthedocs.io/items/FIRFilter.html)

##### Odabir zasebnih graničnih frekvencija za filter (Budući da su razine buke različite za svaki signal)
<img style="float: left; display:block;" src="cutoff frequency.png" />
<br>
- Po tablici se vidi da je za svakog studenta napravljen groupby njegovih očitanja za sva tri testa

#### 2. korak - Izdvajanje značajki i klasifikacija
##### Vektor y 
- vektor svih uzoraka za svakog studenta za određeni ispit

#### Ekstrackija tri segmenta iz svakog trenda
- Uzimaju se segmenti sa početka, sredine i kraja testa (zbog pretpostavke oscilacija stresa tijekom vremena pisanja testa)
- Početni stres - osobni osjećaj sudionika da nije pripremljen
- Krajnji stres - zabrinutost u vezi s preostalim vremenom, umor, potreba za dovršavanjem ispita
- Tw - trajanje segmenta
- T - ukupno trajanje signala

<img style="float: left; display:block;" src="feature extraction.png" />
<br>
- Što se tiće značajki, unutar svakog prozora se izračunava srednja vrijednost i varijanca SC linije trenda

##### Matematički

###### Računjanje segmenata sa Tw:
- $y_{i:j}$ označava vektor koji se sastoji od uzoraka između i, j indeksa

<img style="float: left; display:block;" src="segmenti.png" />

##### Računanje srednje vrijednosti i varijance
- q € {start, mid, end}

<img style="float: left; display:block;" src="mean.png" />

##### Značajka omjera (ratio feature)
- Kako bi vidjeli kako su razine stresa rasle/padale tijekom sredine ispita u usporedbi sa početkom ili krajem ispita

<img style="float: left; display:block;" src="ratio.png" />

##### Difference signal
- informacija koliko se brzo/sporo povečavalo/smanjivalo uzbuđenje tijekom ispita

<img style="float: left; display:block;" src="ydiff.png" />

#### Finalni vektor značajki
$x = [ \mu_{start}, \mu_{mid}, \mu_{end}, \sigma^{2}_{start} \sigma^{2}_{mid}, \sigma^{2}_{end}, \rho, \mu_{diff}, \sigma^{2}_{diff}]$

#### Klasifikacija
- Značajke se uzimaju za $T_{w} = 5, 15, 30 min$
- Treniraju se zasebni klasifikatori za svaki $T_{w}$ te za svaki ispit
- Svakom vektoru značajki se dodaje oznaka visoke ili niske ocjene na temelju toga je li sudionik postigao rezultat >= 80%

- Koriste se klasifikatori iz aplikacije MATLAB 2017b sa defaultnim postavkama
- Procjena izvedbe klasifikatora se vrši sa 10-fold cross-validation

#### Rezultati

<img style="float: left; display:block;" src="rezultati1.png" />

In [9]:
import numpy as np
import pandas as pd
import os
import glob
import matplotlib.pyplot as plt
import seaborn as sns

In [10]:
path = os.getcwd()
s1f = pd.read_csv(path+'\\Data\\S01\\Final\\EDA.csv')

In [89]:
def read_student_eda_files(student):
    csv_files = glob.glob(path+'\\Data\\'+student+'\\*\\EDA.csv')
    
    midterm1_df = pd.read_csv(csv_files[0])
    midterm2_df = pd.read_csv(csv_files[1])
    final_df = pd.read_csv(csv_files[2])
    
    midterm1_df = midterm1_df.iloc[1:]
    midterm2_df = midterm2_df.iloc[1:]
    final_df = final_df.iloc[1:]
    
    return midterm1_df, midterm2_df, final_df

In [127]:
students = ['S01', 'S02', 'S03', 'S04', 'S05', 'S06', 'S07', 'S08', 'S09', 'S10']
cutoff_frequencies = {
    "S01": {"midterm_1": 0.002, "midterm_2": 0.002, "final": 0.002},
    "S02": {"midterm_1": 0.002, "midterm_2": 0.001, "final": 0.002},
    "S03": {"midterm_1": 0.0002, "midterm_2": 0.0002, "final": 0.001},
    "S04": {"midterm_1": 0.002, "midterm_2": 0.002, "final": 0.002},
    "S05": {"midterm_1": 0.001, "midterm_2": 0.001, "final": 0.001},
    "S06": {"midterm_1": 0.002, "midterm_2": 0.002, "final": 0.002},
    "S07": {"midterm_1": 0.001, "midterm_2": 0.0002, "final": 0.0002},
    "S08": {"midterm_1": 0.0002, "midterm_2": 0.002, "final": 0.002},
    "S09": {"midterm_1": 0.002, "midterm_2": 0.002, "final": 0.002},
    "S10": {"midterm_1": 0.002, "midterm_2": 0.002, "final": 0.002}
}
sample_rate = 4
s1f = s1f.iloc[1:]

In [128]:
def get_all_students_eda_dfs(students):
    eda_students_dataframes = {}
    for student in students:
        student_dict = {}
        m1,m2,f = read_student_eda_files(student)
        student_dict['midterm_1'] = m1
        student_dict['midterm_2'] = m2
        student_dict['final'] = f
        eda_students_dataframes[student] = student_dict
        student_dict = {}
    
    return eda_students_dataframes

In [129]:
def get_final_vector(data, Tw, T, sample_rate):
    mod_Tw = Tw*60*sample_rate
    mod_T = T*60*sample_rate
    Ystart = data[0:mod_Tw]
    Ymid = data[(int(mod_T/2) - int(mod_Tw/2)) : (int(mod_T/2) + int(mod_Tw/2))]
    Yend = data [(mod_T - mod_Tw) : mod_T]

    mi_start = np.mean(Ystart)
    mi_mid = np.mean(Ymid)
    mi_end = np.mean(Yend)
    
    si_start = np.var(Ystart)
    si_mid = np.var(Ymid)
    si_end = np.var(Yend)
    
    ro = mi_mid / (mi_start + mi_end)
    
    diff = np.array(Yend) - np.array(Ystart)
    mi_diff = np.mean(diff)
    si_diff = np.var(diff)
    
    x = [mi_start, mi_mid, mi_end, si_start, si_mid, si_end, ro, mi_diff, si_diff]
    
    return x

In [105]:
print(get_final_vector(s1f.values, 15, 90, 4))

[0.022855402222222224, 0.17230403666666666, 0.1913757027777778, 2.455568015439506e-06, 0.020482264257274212, 4.465232180499228e-05, 0.8042904725094269, 0.16852030055555556, 5.379694489688861e-05]


In [150]:
from scipy.signal import butter, filtfilt

def lowpass_butter_filter(data, cutoff, sampling_frequency, order=5):
    nyq = 0.5 * sampling_frequency
    normal_cutoff = cutoff / nyq
    b, a = butter(order, normal_cutoff, btype='low', analog=False)
    y = filtfilt(b, a, data, axis=0)
    
    return pd.DataFrame(y, columns=[data.columns[0]])

In [206]:
import copy

def apply_filter_to_all_students(all_students_dict):
    filtered_dict = copy.deepcopy(all_students_dict)
    for student in all_students_dict:
        for test in all_students_dict[student]:
            filtered_dict[student][test] = lowpass_butter_filter(all_students_dict[student][test], cutoff_frequencies[student][test], 4.000, 4)   
    
    return filtered_dict

In [207]:
students_dict = get_all_students_eda_dfs(students)

In [208]:
filtered_students_dict = apply_filter_to_all_students(students_dict)

In [209]:
for s in students_dict:
    for t in students_dict[s]:
        res = "{:<20}  |  {:<20}".format(str(students_dict[s][t].values.mean()), str(filtered_students_dict[s][t].values.mean()))
        print("{} - {:<10} :   ".format(s,t) + res)
    print("-"*80)

S01 - midterm_1  :   0.10292479984398711   |  0.10287115715420407 
S01 - midterm_2  :   0.32590895987654317   |  0.3258099451001292  
S01 - final      :   0.11658823646282328   |  0.11647831526113792 
--------------------------------------------------------------------------------
S02 - midterm_1  :   0.07829630200501253   |  0.07825633169272297 
S02 - midterm_2  :   0.16454142616931614   |  0.16425294799303297 
S02 - final      :   0.12559881131158168   |  0.12553709809330751 
--------------------------------------------------------------------------------
S03 - midterm_1  :   0.16966423930134386   |  0.1723609049104544  
S03 - midterm_2  :   0.23662251768607223   |  0.2745560951692351  
S03 - final      :   0.5719971969985359    |  0.5731622922727496  
--------------------------------------------------------------------------------
S04 - midterm_1  :   0.5262481283259511    |  0.5262244275751149  
S04 - midterm_2  :   0.3399011036642723    |  0.3398744013782394  
S04 - final      :  